# 🤖 Aula 4: Criando um Chatbot com Memória

Neste notebook, vamos construir um chatbot que lembra informações do usuário, com foco em conceitos técnicos, matemáticos e exemplos de código mais robustos.

## 📌 O que faremos:
1. Criar um histórico de mensagens para armazenar contexto.
2. Implementar um chatbot que lembra do usuário ao longo da conversa.
3. Explorar técnicas de retenção de memória e gerenciamento de contexto.
4. Testar e melhorar a retenção de memória.


## 🧠 Conceitos Técnicos e Matemáticos

### Memória em AI Agents
A memória em AI Agents é crucial para manter o contexto de uma conversa. Ela pode ser implementada de várias formas:
- **Memória de Curto Prazo:** Armazena as últimas interações para manter o contexto atual.
- **Memória de Longo Prazo:** Armazena informações importantes para uso futuro, como o nome do usuário ou preferências.

### Gerenciamento de Contexto
O contexto é gerenciado através de uma **sequência de mensagens**, onde cada mensagem tem um papel (ex.: `system`, `user`, `assistant`). A API do OpenAI usa essa sequência para gerar respostas contextualizadas.

A probabilidade de uma resposta é dada por:

$$ P(r \mid c) = \prod_{i=1}^n P(r_i \mid r_1, r_2, \dots, r_{i-1}, c) $$

Onde:
- **$r$**: Resposta gerada.
- **$c$**: Contexto (histórico de mensagens).
- **$r_i$**: Palavra na posição $i$ da resposta.

### Limitação de Memória
Para evitar o acúmulo infinito de mensagens, é comum limitar o histórico a um número fixo de interações. Isso é feito removendo as mensagens mais antigas do contexto.


In [ ]:
# Instale a biblioteca OpenAI se ainda não estiver instalada
!pip install openai

In [ ]:
import openai

# Defina sua chave de API (substitua 'SUA_CHAVE_AQUI' pela chave real)
openai.api_key = "SUA_CHAVE_AQUI"

In [ ]:
# Criando um histórico de mensagens para armazenar contexto
historico = [{"role": "system", "content": "Você é um assistente amigável."}]

def chatbot_memoria(pergunta):
    historico.append({"role": "user", "content": pergunta})
    resposta = openai.ChatCompletion.create(
        model="gpt-4",
        messages=historico
    )
    mensagem = resposta["choices"][0]["message"]["content"]
    historico.append({"role": "assistant", "content": mensagem})
    return mensagem

# Teste rápido
print(chatbot_memoria("Meu nome é João."))
print(chatbot_memoria("Qual é o meu nome?"))

In [ ]:
# Melhorando a retenção de contexto com um histórico mais longo
def chatbot_memoria_aprimorado(pergunta, max_historico=5):
    # Mantendo apenas as últimas N interações para evitar acúmulo infinito
    if len(historico) > max_historico * 2:
        historico.pop(1)
        historico.pop(1)
    
    historico.append({"role": "user", "content": pergunta})
    resposta = openai.ChatCompletion.create(
        model="gpt-4",
        messages=historico
    )
    mensagem = resposta["choices"][0]["message"]["content"]
    historico.append({"role": "assistant", "content": mensagem})
    return mensagem

# Teste aprimorado
print(chatbot_memoria_aprimorado("Meu nome é Ana."))
print(chatbot_memoria_aprimorado("O que eu te disse antes?"))

## 🛠️ Exemplo Prático: Chatbot com Memória de Longo Prazo

Vamos criar um chatbot que armazena informações importantes (como o nome do usuário) em um banco de dados simples para memória de longo prazo.


In [ ]:
import sqlite3

# Criando um banco de dados simples para memória de longo prazo
conn = sqlite3.connect("chatbot_memoria.db")
cursor = conn.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS memoria (chave TEXT, valor TEXT)")
conn.commit()

class ChatbotMemoriaLongoPrazo:
    def __init__(self):
        self.historico = [{"role": "system", "content": "Você é um assistente amigável."}]

    def salvar_memoria(self, chave, valor):
        cursor.execute("INSERT INTO memoria (chave, valor) VALUES (?, ?)", (chave, valor))
        conn.commit()

    def recuperar_memoria(self, chave):
        cursor.execute("SELECT valor FROM memoria WHERE chave = ?", (chave,))
        resultado = cursor.fetchone()
        return resultado[0] if resultado else None

    def perguntar(self, pergunta):
        if "meu nome é" in pergunta.lower():
            nome = pergunta.split("meu nome é")[1].strip()
            self.salvar_memoria("nome_usuario", nome)
            self.historico.append({"role": "user", "content": pergunta})
            resposta = f"Prazer em conhecê-lo, {nome}! Como posso ajudar?"
            self.historico.append({"role": "assistant", "content": resposta})
            return resposta

        nome_usuario = self.recuperar_memoria("nome_usuario")
        if nome_usuario:
            pergunta = f"{nome_usuario}, {pergunta}"

        self.historico.append({"role": "user", "content": pergunta})
        resposta = openai.ChatCompletion.create(
            model="gpt-4",
            messages=self.historico
        )
        mensagem = resposta["choices"][0]["message"]["content"]
        self.historico.append({"role": "assistant", "content": mensagem})
        return mensagem

# Testando o chatbot com memória de longo prazo
bot = ChatbotMemoriaLongoPrazo()
print(bot.perguntar("Meu nome é João."))
print(bot.perguntar("Qual é o meu nome?"))
print(bot.perguntar("O que é um AI Agent?"))

## 🎯 Desafios Técnicos

1. **Desafio 1:** Modifique o chatbot para lembrar mais detalhes da conversa, como preferências ou hobbies do usuário.
2. **Desafio 2:** Implemente um sistema de **limpeza de memória** que remove informações antigas após um período de tempo.
3. **Desafio 3:** Crie um chatbot que usa embeddings para comparar perguntas anteriores e sugerir respostas baseadas em interações passadas.

Compartilhe suas soluções no fórum do curso!